# **Importing the dependencies**

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report


# **Data Collection & Preprocessing**

In [8]:
data=pd.read_csv('Parkinsson disease.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Parkinsson disease.csv'

In [6]:
data.sample(5)

NameError: name 'data' is not defined

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
# distribution of target Variable
data['status'].value_counts()

In [ ]:
# grouping the data based on the target variable
data.groupby('status').mean(numeric_only=True)

In [ ]:
data = data.drop('name', axis = 1)


# **EDA**

In [ ]:
#heatmap
plt.figure(figsize=(30,30))
sns.heatmap(data.corr(), annot = True, cmap= "coolwarm")

In [ ]:
#barplot
data.corr()['status'][:-1].sort_values().plot(kind='bar')


# **Neural Network model**

In [ ]:
for col in data:
     print (f'{col}:{data[col].unique()}')

In [ ]:
scaler=MinMaxScaler()
cols_to_scale=['MDVP:Fo(Hz)','MDVP:Fhi(Hz)','MDVP:Flo(Hz)','MDVP:Jitter(Abs)','HNR','spread1','D2']
data[cols_to_scale]=scaler.fit_transform(data[cols_to_scale])

In [ ]:
data.columns

In [ ]:
X=data.drop('status',axis=1)
y=data['status']

In [ ]:
from imblearn.over_sampling import SMOTE

In [7]:
smote=SMOTE(sampling_strategy='minority')
X_sm,y_sm=smote.fit_resample(X,y)
y_sm.value_counts()

NameError: name 'SMOTE' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm,test_size=0.2,random_state=15,stratify=y_sm)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(20, input_dim=22, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=500)
    else:
        model.fit(X_train, y_train, epochs=500, class_weight = weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)


In [ ]:
cm=tf.math.confusion_matrix(labels=y_test,predictions=y_preds)
sns.heatmap(cm,annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(accuracy_score(y_test,y_preds))
